<a href="https://colab.research.google.com/github/Sandwhaletree/2023.05_Tibame/blob/main/%E9%80%B2%E9%9A%8EDL_%E6%9D%8E%E6%99%BA%E6%8F%9A_Class/007_YOLOv8_BCCD_0718.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from glob import glob #在目錄中查找符合特定規則的文件路徑名
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import os
from tqdm.auto import tqdm #tqdm是一個快速，可擴展的Python進度條
from sklearn.model_selection import train_test_split #train_test_split函數用於將數據集分割為訓練集和測試集

import torch #開源的機器學習庫，提供了許多深度學習的功能
from torch import nn #nn模組提供了許多用於建立神經網路的類和函數
import torchvision #torchvision是一個處理圖像和視覺的torch包，提供了許多視覺圖像處理的工具和數據集
from torchvision import models, transforms #models提供了許多預訓練的模型，如ResNet, VGG等。transforms提供了許多圖像預處理的方法
from torchsummary import summary #torchsummary提供了一種用於顯示模型結構的方法

print("PyTorch Version: ", torch.__version__) #PyTorch版本2.0.1+cu118
print("Torchvision Version: ", torchvision.__version__) #Torchvision版本0.15.2+cu118

In [ ]:
#設定模型的存儲路徑和選擇運算裝置
MODEL_PATH = 'model.pth' #'model.pth': 這行程式碼是在設定模型的存儲路徑，這裡將模型的存儲路徑設定為當前目錄下的'model.pth'文件
device = 'cuda' if torch.cuda.is_available() else 'cpu' #選擇運算裝置，如有gpu裝置設為cuda，否則為cpu
print(f'device: {device}')

#### Download Dataset

In [ ]:
# Download dataset from GoogleDrive
!pip install --upgrade gdown #gdown是一個可以從Google Drive下載文件的命令行工具
!gdown --fuzzy 1kIef9G5Og7VgDPEHt_vIJMTKHdC7_0Kq --output "./NEU_defect.zip" #用gdown工具從Google Drive下載數據集 #這裡的1kIef9G5Og7VgDPEHt_vIJMTKHdC7_0Kq是Google Drive文件的ID，--output "./NEU_defect.zip"指定了下載的文件的存儲路徑和名稱
!unzip -q "./NEU_defect.zip" #-q選項表示在解壓縮時不輸出任何信息，"./NEU_defect.zip"是要解壓縮的zip文件的路徑

In [ ]:
IMG_SIZE = 200 # 128
class_names = ['PS', 'Sc', 'RS', 'In', 'Cr', 'Pa']
class_map = {name: i for i, name in enumerate(class_names)}
print(class_map)
NUM_CLASS = len(class_names)

In [ ]:
# search .bmp file paths
paths = glob("./NEU_defect/*.bmp")
print("number of samples", len(paths))

#### Dataset & Dataloader

In [ ]:
class NEUDataset(torch.utils.data.Dataset):
    def __init__(self, img_paths, img_size, cls_map, transform):
        self.img_paths = img_paths
        self.img_size = img_size
        self.cls_map = cls_map
        self.transform = transform

    def __len__(self):
        """number of samples"""
        return len(self.img_paths)

    def __getitem__(self, idx):
        # Read img
        path = self.img_paths[idx] # get img path
        img = Image.open(path).convert('RGB')

        # transform img
        img = self.transform(img) #資料前處理

        # Read class index
        cls_name = path.split('/')[-1].split('_')[0]
        cls_idx = self.cls_map[cls_name]
        cls_idx = torch.tensor(cls_idx, dtype=torch.int64)
        return img, cls_idx

In [ ]:
# split dataset
train_paths, val_paths = train_test_split(paths,
    random_state=556,
    test_size=0.2) #切20%資料

len(train_paths), len(val_paths)

In [ ]:
from torchvision.models import resnet50, ResNet50_Weights

# Preprocess Transform
transform =  ResNet50_Weights.DEFAULT.transforms() #DEFAULT找預設最好的參數給你

# Build Dataset
train_ds = NEUDataset(train_paths, IMG_SIZE, class_map,
                      transform)
val_ds = NEUDataset(val_paths, IMG_SIZE, class_map,
                    transform)

In [ ]:
transform #先resize在切

In [ ]:
# Build dataloader
BS = 32
train_loader = torch.utils.data.DataLoader(train_ds, BS, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_ds, BS)

In [ ]:
# Visulize sample
idx = np.random.randint(0, len(train_ds))
img, cls = train_ds[idx]

img_normalized = img.permute(1, 2, 0)
plt.imshow(img_normalized)
plt.show()

# Convert back to original value
img_raw = img.numpy().transpose(1, 2, 0) # (3, 256, 256) -> (256, 256, 3)
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
img_raw = std * img_raw + mean
img_raw = np.clip(img_raw, 0, 1)
plt.imshow(img_raw)
plt.show()

print('Class:', cls)

#### Build Model

Pick your favorite models: https://pytorch.org/vision/stable/models.html

In [ ]:
model = models.resnet50(weights=ResNet50_Weights.DEFAULT) #ResNet50_Weights.DEFAULT與之前不同的地方
print(model)

In [ ]:
# replace classifier
num_features = model.fc.in_features # len of feature vectors

# # Freeze model
# for param in model.parameters():
#     param.requires_grad = False

# Replace classifier
model.fc = nn.Linear(num_features, 6)
print(model.fc)


In [ ]:
summary(model.to(device), (3, 224, 224))

#### Training

In [ ]:
def train_epoch(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset) # number of samples
    num_batches = len(dataloader) # batches per epoch

    model.train() # to training mode.
    epoch_loss, epoch_correct = 0, 0
    for batch_i, (x, y) in enumerate(tqdm(dataloader, leave=False)):
        x, y = x.to(device), y.to(device) # move data to GPU

        # zero the parameter gradients
        optimizer.zero_grad()

        # Compute prediction loss
        pred = model(x)
        loss = loss_fn(pred, y)

        # Optimization by gradients
        loss.backward() # backpropagation to compute gradients
        optimizer.step() # update model params

        # write to logs
        epoch_loss += loss.item() # tensor -> python value
        # (N, Class)
        epoch_correct += (pred.argmax(dim=1) == y).sum().item()

    # return avg loss of epoch, acc of epoch
    return epoch_loss/num_batches, epoch_correct/size


def test_epoch(dataloader, model, loss_fn):
    size = len(dataloader.dataset) # number of samples
    num_batches = len(dataloader) # batches per epoch

    model.eval() # model to test mode.
    epoch_loss, epoch_correct = 0, 0

    # No gradient for test data
    with torch.no_grad():
        for batch_i, (x, y) in enumerate(dataloader):
            x, y = x.to(device), y.to(device)

            # Compute prediction loss
            pred = model(x)
            loss = loss_fn(pred, y)

            # write to logs
            epoch_loss += loss.item()
            epoch_correct += (pred.argmax(1) == y).sum().item()

    return epoch_loss/num_batches, epoch_correct/size

```python
from torchvision.models import resnet50, ResNet50_Weights

resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
resnet50(weights=ResNet50_Weights.DEFAULT)

# Strings are also supported
resnet50(weights="IMAGENET1K_V2")

# No weights - random initialization
resnet50(weights=None)

# Old version
# pretrained weights:
resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
resnet50(weights="IMAGENET1K_V1")
resnet50(pretrained=True)  # deprecated
resnet50(True)  # deprecated

# no weights:
resnet50(weights=None)
resnet50()
resnet50(pretrained=False)  # deprecated
resnet50(False)  # deprecated
```

In [ ]:
def train(pretrained, freeze=False): #要不要做pretrained  要不要做freeze
    # Model
    if pretrained:
        model = models.resnet50(weights=ResNet50_Weights.DEFAULT)
    else:
        model = models.resnet50(weights=None)

    if freeze: #要不要做freeze
        # Freeze model
        for param in model.parameters():
            param.requires_grad = False
    # Replace classifier
    num_features = model.fc.in_features # len of feature vectors
    model.fc = nn.Linear(num_features, 6)
    model = model.to(device)

    loss_fn = nn.CrossEntropyLoss()
    # lower learning rate for finetuning
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    EPOCHS = 10
    logs = {
        'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []
    }
    for epoch in tqdm(range(EPOCHS)):
        train_loss, train_acc = train_epoch(train_loader, model, loss_fn, optimizer)
        val_loss, val_acc = test_epoch(val_loader, model, loss_fn)

        print(f'EPOCH: {epoch:04d} \
        train_loss: {train_loss:.4f}, train_acc: {train_acc:.3f} \
        val_loss: {val_loss:.4f}, val_acc: {val_acc:.3f} ')

        logs['train_loss'].append(train_loss)
        logs['train_acc'].append(train_acc)
        logs['val_loss'].append(val_loss)
        logs['val_acc'].append(val_acc)

    # plot result
    plt.figure(figsize=(6, 3))
    plt.subplot(1, 2, 1)
    plt.title('Loss')
    plt.plot(logs['train_loss'])
    plt.plot(logs['val_loss'])
    plt.subplot(1, 2, 2)
    plt.title('Acc.')
    plt.plot(logs['train_acc'])
    plt.plot(logs['val_acc'])
    plt.show()

    # Save model
    torch.save(model, MODEL_PATH)

In [ ]:
train(pretrained=False)
train(pretrained=True)
train(pretrained=True, freeze=True)

#### Ref:

[Official: Transfer learning tutorials](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html)

[Official: Finetuning torchvision models tutorial](https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html#load-data)

[Accessing-and-modifying-different-layers-of-a-pretrained-model-in-pytorch](https://github.com/mortezamg63/Accessing-and-modifying-different-layers-of-a-pretrained-model-in-pytorch)

#### Data Process